In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import glob
import imageio.v3 as iio
import numpy as np
import os
import pandas as pd
import pickle

In [ ]:
ROOT_FILE_PATH = "/content/drive/My Drive/2023-Machine-Learning-Dataset";

SUBJ_LIST = ["subj01", "subj02", "subj03", "subj04", "subj05", "subj06", "subj07", "subj08"];

def ReadNpyFile(folderdir):
  retdic = {};
  for path in glob.glob(folderdir + "/*.npy"):
    retdic[path.split("/")[-1]] = np.load(path, allow_pickle = True);
  return retdic;

def ReadPngFile(folderdir):
  retdic = {};
  for path in glob.glob(folderdir + "/*.png"):
    retdic[path.split("/")[-1]] = iio.imread(path);
  return retdic;

def read_subject(subject):
  retdic = {};
  fp = "/".join([ROOT_FILE_PATH, subject]);
  retdic["training_img"] = ReadPngFile(os.path.join(fp, "training_split", "training_images"))
  retdic["training_fmri"] = ReadNpyFile(os.path.join(fp, "training_split", "training_fmri"));
  retdic["test_fmri"] = ReadNpyFile(os.path.join(fp, "test_split", "test_fmri"));
  retdic["roi_masks"] = ReadNpyFile(os.path.join(fp, "roi_masks"));
  return retdic;

In [ ]:
DATA = read_subject(SUBJ_LIST[0])  #Requires 75 s approx.

# Todo for preprocessing
- Training / Validation split
- Documentation for all dimensions of all variables
- Category for all images (array with size of all categories value 0/1)
- Images in rgb and grey scale